##Анализ токсичности комментариев

## Загрузка набора данных

Загрузка наборов предварительно обученных вложений слов

In [ ]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip -O glove.twitter.27B.zip

--2020-08-10 03:26:21--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2020-08-10 03:26:21--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2020-08-10 03:26:22--  http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [appli

In [ ]:
!unzip glove.twitter.27B.zip

Archive:  glove.twitter.27B.zip
  inflating: glove.twitter.27B.25d.txt  
  inflating: glove.twitter.27B.50d.txt  
  inflating: glove.twitter.27B.100d.txt  
  inflating: glove.twitter.27B.200d.txt  


Распоковка датасета

In [ ]:
!unzip jigsaw-toxic-comment-classification-challenge.zip -d /content/dataset

Archive:  jigsaw-toxic-comment-classification-challenge.zip
  inflating: /content/dataset/sample_submission.csv.zip  
  inflating: /content/dataset/test.csv.zip  
  inflating: /content/dataset/test_labels.csv.zip  
  inflating: /content/dataset/train.csv.zip  


In [ ]:
!unzip /content/dataset/test.csv.zip
!unzip /content/dataset/train.csv.zip
!unzip /content/dataset/test_labels.csv.zip

Archive:  /content/dataset/test.csv.zip
  inflating: test.csv                
Archive:  /content/dataset/train.csv.zip
  inflating: train.csv               
Archive:  /content/dataset/test_labels.csv.zip
  inflating: test_labels.csv         


## Начало

In [ ]:
import numpy as np
import pandas as pd

from scipy import sparse

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D, GRU
from keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
%matplotlib inline

In [ ]:
# векторы слов GLoVe, чтобы получить предварительно обученные вложения слов
embed_size = 100 
# максимальное количество уникальных слов 
max_features = 10000
# максимальное количество слов из отзыва для использования
maxlen = 100

## Считываем данные

In [ ]:
train = pd.read_csv('train.csv')

test_comments = pd.read_csv('test.csv')
test_labels = pd.read_csv('test_labels.csv')
test = pd.merge(test_comments, test_labels, on='id') #тестовая выборка разбита на 2 файла 
test = test[test['toxic']!=-1]
train.shape, test.shape, train.head()

((159571, 8), (63978, 8),                  id  ... identity_hate
 0  0000997932d777bf  ...             0
 1  000103f0d9cfb60f  ...             0
 2  000113f07ec002fd  ...             0
 3  0001b41b1c6bb37e  ...             0
 4  0001d958c54c6e35  ...             0
 
 [5 rows x 8 columns])

Вам предоставлено большое количество комментариев Википедии, которые были отмечены человеческими оценщиками как токсичное поведение. Типы токсичности:

toxic (токсичный)

severe_toxic (существенно токсичный)

obscene (обсценная лексика)

threat (угроза)

insult (оскорбление)

identity_hate (личная ненависть)


Перемешаем наши данные

In [ ]:
# установите frac = 1., чтобы использовать весь образец
train_samp = train.sample(frac = 1., random_state = 42)
test_samp = test.sample(frac = 1., random_state = 42)
train_samp.shape, test_samp.shape

((159571, 8), (63978, 8))

Создаем вектор ответов y

In [ ]:
class_names = ['toxic',	'severe_toxic',	'obscene',	'threat',	'insult',	'identity_hate']
y = train_samp[class_names].values

Обучаем токинезатор слов

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_samp['comment_text'].values))
X_train = tokenizer.texts_to_sequences(train_samp['comment_text'].values)
X_test = tokenizer.texts_to_sequences(test_samp['comment_text'].values)
x_train = pad_sequences(X_train, maxlen = maxlen)
x_test = pad_sequences(X_test, maxlen = maxlen)

# сохранение токинезатора слов
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Загружаем модели заранее обученых английских  слов.

Слой Keras Embedding может также использовать встраивание слова, изученное в другом месте.

В области обработки естественного языка принято изучать, сохранять и делать свободно доступные вложения слов.

Например, исследователи, стоящие за методом GloVe, предоставляют набор предварительно обученных вложений слов на своем веб-сайте, выпущенных по лицензии общественного достояния. Увидеть:

GloVe: глобальные векторы для представления слов
Самый маленький пакет вложений 822Mb, называетсяglove.6B.zip«. Он был обучен на наборе данных из одного миллиарда токенов (слов) со словарем в 400 тысяч слов. Существует несколько различных размеров встраиваемых векторов, в том числе 50, 100, 200 и 300 размеров.

Вы можете скачать эту коллекцию вложений, и мы можем посеять KerasВстраиваниеслой с весами из предварительно обученного встраивания слов в ваш тренировочный набор данных.

In [ ]:
# # Файл патч
# embedding_file = '/content/glove.twitter.27B.100d.txt'

# # читать во вложениях
# def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
# embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(embedding_file))

In [ ]:
# word_index = tokenizer.word_index

# nb_words = min(max_features, len(word_index))
# # создаем матрицу нулей правильных размеров 
# embedding_matrix = np.zeros((nb_words, embed_size))
# missed = []
# for word, i in word_index.items():
#     if i >= max_features: break
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector
#     else:
#         missed.append(word)
#print(len(missed))
#print(missed[0:10])
#print(missed[len(missed)-10:len(missed)])

## Создаем и обучаем модель

Создаем модель

In [ ]:
inp = Input(shape = (maxlen,))
#x = Embedding(max_features, embed_size, weights = [embedding_matrix], trainable = True)(inp)
x = Embedding(max_features, embed_size)(inp) # без использования обученого словаря
x = SpatialDropout1D(0.5)(x)
x = Bidirectional(LSTM(40, return_sequences=True))(x)
x = Bidirectional(GRU(40, return_sequences=True))(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
conc = concatenate([avg_pool, max_pool])
outp = Dense(6, activation = 'sigmoid')(conc)

model = Model(inputs = inp, outputs = outp)
#Остановка обучения и сохранения лучших весов
earlystop = EarlyStopping(monitor = 'val_accuracy', min_delta = 0, patience = 3) # patience - это сколько эпох ждать, чтобы увидеть, улучшится ли val_loss снова.
checkpoint = ModelCheckpoint(monitor = 'val_loss', save_best_only = True, filepath = 'wiki_lstm_gru_weights.hdf5',verbose=1)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.fit(x_train, y, batch_size = 256, epochs = 5, validation_split = .1,
          callbacks=[earlystop, checkpoint])
# Если хотите сохранить модель 

Epoch 1/5
561/561 [==============================] - ETA: 0s - loss: 0.1080 - accuracy: 0.9889
Epoch 00001: val_loss improved from inf to 0.05167, saving model to yelp_lstm_gru_weights.hdf5
561/561 [==============================] - 70s 125ms/step - loss: 0.1080 - accuracy: 0.9889 - val_loss: 0.0517 - val_accuracy: 0.9938
Epoch 2/5
561/561 [==============================] - ETA: 0s - loss: 0.0491 - accuracy: 0.9941
Epoch 00002: val_loss improved from 0.05167 to 0.05027, saving model to yelp_lstm_gru_weights.hdf5
561/561 [==============================] - 69s 122ms/step - loss: 0.0491 - accuracy: 0.9941 - val_loss: 0.0503 - val_accuracy: 0.9938
Epoch 3/5
561/561 [==============================] - ETA: 0s - loss: 0.0457 - accuracy: 0.9935
Epoch 00003: val_loss improved from 0.05027 to 0.04882, saving model to yelp_lstm_gru_weights.hdf5
561/561 [==============================] - 68s 122ms/step - loss: 0.0457 - accuracy: 0.9935 - val_loss: 0.0488 - val_accuracy: 0.9938
Epoch 4/5
561/561 [=

## Тест модели

загрузка модели

In [ ]:
from tensorflow.keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

maxlen = 100
model=load_model("wiki_lstm_gru_weights.hdf5")

In [ ]:
y_test = test_samp[class_names].values

In [ ]:
model.evaluate(x_test, test_samp[class_names].values, verbose = 1, batch_size=1024)

63/63 [==============================] - 4s 60ms/step - loss: 0.0762 - accuracy: 0.9964


[0.07622984796762466, 0.9963580965995789]

##Проверка на своем примере

In [ ]:
comment = "X-BOX 360 SUKCS BIG BUMM AND LIKES IT UP THE ASS"

In [ ]:
sequence = tokenizer.texts_to_sequences([comment])

In [ ]:
data = pad_sequences(sequence, maxlen=maxlen)

In [ ]:
result = model.predict(data)

In [ ]:
result

array([[0.9292261 , 0.0121293 , 0.72989225, 0.00292371, 0.3505534 ,
        0.02182196]], dtype=float32)

Правильный ответ:

[1, 0, 1, 0, 1, 0]